In [1]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import pandas as pd
import spacy
import torch
from google.cloud import bigquery
import os

In [ ]:

os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "../ETL/credencial.json"

client = bigquery.Client()

query = f"""
        SELECT *
        FROM `pure-hue-399113.restaurant_staging_dataset.restaurants_dim`
        """
        """
        SELECT tp.*, ts1.*, ts2.*
        FROM tabla_principal tp
        LEFT JOIN tabla_secundaria1 ts1 ON tp.id = ts1.id_principal
        LEFT JOIN tabla_secundaria2 ts2 ON tp.id = ts2.id_principal
        WHERE tp.id = tu_id_aqui;
        """

query_job = client.query(query)
results = query_job.result()

rows = []
for row in results:
    rows.append(row)

results_df = pd.DataFrame(data=[list(x.values()) for x in rows], columns=list(rows[0].keys()))
results_df